In [3]:
#import random
import datetime
import time
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
#import jieba
#import jieba.analyse
#import gensim
#from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split

%matplotlib notebook

plt.rcParams['font.sans-serif']=['SimHei']

In [4]:
from fastFM import als
from scipy import sparse
import apriori as ap
import freq_patt_tree as fpt
import util

In [7]:
import imp
fpt = imp.reload(fpt)
ap = imp.reload(ap)
util = imp.reload(util)

In [ ]:
support_dict = ap.load_support_dict('out_large_itemsets.csv')
ap.compute_recom_rules(support_dict, 0.8)

In [9]:
time_s = time.time()
large_itemsets, recomm_rules = ap.run(util.yield_actionsets_from_file('../data/uid_sid_sparse.csv'), 20, 0.8,
                                      max_large_item_len=None, use_fp_tree=True, output_support_only=False)
time_e = time.time()
print('time diff:', time_e - time_s)
ap.dump(large_itemsets, recomm_rules, to_file_only=True)

time diff: 13.085367918014526


In [ ]:
time_s = time.time()
root, header = fpt.create_tree(util.yield_actionsets_from_file('../data/uid_sid_sample.csv'), 2)
large_itemsets = fpt.compute_large_itemsets(root, header, 2)
time_e = time.time()
print('time diff:', time_e - time_s)
print(large_itemsets)
root.dump()

In [ ]:
util = imp.reload(util)
util.create_itemset_from_file('../data/sample_simple.csv')

In [ ]:
util = imp.reload(util)
list(util.yield_itemsets_from_file('../data/sample_simple.csv'))

In [ ]:
util = imp.reload(util)
util.create_itemset_from_dataframe(pd.read_csv('../data/sample_simple.csv', header=None))

In [ ]:
util = imp.reload(util)
itemvec_dict = util.create_onehot_vector(util.create_itemset_from_file('../data/sample_simple.csv'))

In [ ]:
(itemvec_dict["beer"] + itemvec_dict["mango"]).todense()

In [ ]:
uservec = create_onehot_vec(df.index)

In [ ]:
uservec

In [ ]:
mat = create_user_item_matrix(df, uservec, itemvec, mimic="SVD++")
#mat = create_user_item_matrix(df, uservec, itemvec, mimic="MF")
#mat.todense()

In [ ]:
X = mat
y = np.ones(mat.shape[0])
X_ = sparse.csc_matrix(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [ ]:
fm = als.FMRegression(n_iter=200, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [ ]:
y_pred

In [ ]:
### apriori/fpgrowth

In [ ]:
df_puffer = pd.read_excel('180419mark_satisfied_puffer.xlsx')

In [ ]:
dfp = df_puffer.copy()

In [ ]:
df1 = dfp[dfp['mark'] == 1].copy()

In [ ]:
def print_apriori_rules(df_sidlookup, fn):
    def get_sid_info(df, sid):
        return df.loc[sid].iloc[0]['content'] + df.loc[sid].iloc[0]['slots']

    fd = open(fn, 'w')
    for index, row in df_rule.iterrows():
        for sid in row[1].split('|'):
            if sid == '':
                continue
            fd.write("<<< " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
        for sid in row[2].split('|'):
            if sid == '':
                continue
            fd.write(">>> " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
        fd.write('\n')
    fd.close()

In [ ]:
def mark_sid_type(row):
    botid = row['source_type']
    sid = row['sid']
    if botid == "audio_music":
        row['sid'] = "m" + sid
    elif botid == "audio_unicast":
        row['sid'] = "u" + sid
    elif botid == "ai.dueros.bot.short_video":
        row['sid'] = "s" + sid
    elif botid == "ai.dueros.bot.video_on_demand":
        row['sid'] = "v" + sid
    return row

In [ ]:
dfs = df1.apply(mark_sid_type, axis=1)

In [ ]:
df_sidlookup = dfs[['sid', 'slots', 'content', 'nlu']].set_index('sid')
df_sidlookup.to_excel('out_sid_lookup.xlsx')
df_sidlookup.head()

In [ ]:
df1 = df_sidlookup.reset_index()

In [ ]:
len(df1['sid'])

In [ ]:
df_sid = dfs[['uid', 'sid']].groupby('uid').agg(lambda x: ','.join(x))

In [ ]:
df_sid.to_csv('out_uid_sid.csv', index=False, header=None, sep='|')

In [ ]:
import imp
import time
import util
import freq_patt_tree as fpt
fpt = imp.reload(fpt)
root, header = fpt.create_tree(util.yield_actionsets_from_file('../data/uid_sid_sample.csv'), min_sup=2)
support_dict = fpt.compute_large_itemsets(root, header, min_sup=2)
print('len(support_dict):', len(support_dict))
root.dump()

In [ ]:
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(util.yield_actionsets_from_file('../data/uid_sid_sample.csv'),
                                      2, 0.2, use_fp_tree=True, output_support_only=False)
time_e = time.time()
print('time diff:', time_e - time_s)

In [ ]:
import apriori as ap
import imp
import time
import util
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(util.yield_actionsets_from_file('../data/uid_sid_sample.csv'),
                                      2, 0.2, use_fp_tree=False, output_support_only=True)
time_e = time.time()
print('time diff:', time_e - time_s)
ap.dump(large_itemsets, recomm_rules, to_file_only=True)

In [ ]:
import apriori as ap
import imp
import time
import util
ap = imp.reload(ap)
time_s = time.time()
large_itemsets, recomm_rules = ap.run(util.yield_actionsets_from_file('../data/uid_sid_sparse.csv'),
                                      2, 0.2, use_fp_tree=True, output_support_only=True)
time_e = time.time()
print('time diff:', time_e - time_s)
ap.dump(large_itemsets, recomm_rules, to_file_only=True)

In [ ]:
fpt = imp.reload(fpt)
root, header = fpt.create_tree(util.yield_actionsets_from_file('../data/uid_sid_dense_3.csv'), min_sup=2)
root.dump()

In [ ]:
util = imp.reload(util)
len(list(util.create_itemset_from_iterator(
    util.yield_actionsets_from_file('../data/uid_sid_dense_3.csv'))))

In [ ]:
df_rule = pd.read_csv('out_recomm_rules_20%.csv', header=None)

In [ ]:
print_apriori_rules(df_sidlookup, 'out_rules_display.txt')

In [ ]:
### SVD++

In [ ]:
df_sid.head()

In [ ]:
itemset = create_item_set_from_file('out_uid_sid.csv')

In [ ]:
itemvec = create_onehot_vec(itemset)

In [ ]:
len(itemvec)

In [ ]:
itemvec

In [ ]:
uservec = create_onehot_vec(df_sid.index)

In [ ]:
uservec

In [ ]:
df_sid.loc['3F18061186542DB1']

In [ ]:
mat = create_user_item_matrix_compacted_sid(df_sid, uservec, itemvec, mimic="SVD++")

In [ ]:
itemvec['u57751445013']

In [ ]:
X = mat
y = np.ones(mat.shape[0])
X_ = sparse.csc_matrix(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [ ]:
fm = als.FMRegression(n_iter=200, init_stdev=0.1, rank=50, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [ ]:
y_pred

In [ ]:
X_eval = create_user_item_predict_mat_compacted_sid('3F180611863094C1', df_sid, uservec, itemvec, mimic="SVD++")

In [ ]:
y_eval = fm.predict(X_eval)

In [ ]:
y_eval[:200]

In [ ]:
###

In [ ]:
uid = 1
fd1 = open('uid_sid_sample.csv', 'w')
for actionset in util.yield_itemsets_from_file('../data/sample_simple.csv'):
    fd1.write("user" + str(uid) + ",")
    uid += 1
    fd1.write("|".join(actionset) + "\n")
    
fd1.close()

In [ ]:
util = imp.reload(util)
list(util.yield_actionsets_from_file('../data/uid_sid_sample.csv'))